## Parameterized Decorators
When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function. So how do you make a decorator accept other arguments? 

Make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated.

In [2]:
registry = []

def register(func):
    print('running register {}'.format(func))
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register <function f1 at 0x7f4378492d90>
running main()
registry -> [<function f1 at 0x7f4378492d90>]
running f1()


### A Parameterized Registration Decorator

In order to make it easy to enable to disable the function registration performed by register, we'll make it accept an optional `active` parameter which if `False`, skips registering the decorated function. Conceptually, the new `register` function is not a decorator but a decorator factory.

In [3]:
%%writefile registration_param.py
registry = set() # Adding and removing functions is faster

def register(active=True):
    def decorate(func): # This is the actual decorator - it takes a function as an argument
        print('Running register(active=%s) -> decorate(%s)'
             % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

# Now register must be invoked as a function with desired parameters
@register(active=False)
def f1():
    print('running f1()')
    
# Even if no parameters, still need to be a function
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

Writing registration_param.py


In [4]:
import registration_param

Running register(active=False) -> decorate(<function f1 at 0x7f435e7011e0>)
Running register(active=True) -> decorate(<function f2 at 0x7f435e701400>)


In [5]:
registration_param.registry

{<function registration_param.f2()>}

### The Parameterized Clock Decorator
Users may pass a format string to control the output of the decorated function.

In [10]:
%%writefile clockdeco_param.py
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): # decorator factory
    def decorate(func): # decoractor
        def clocked(*_args): # clocked wraps the decorated function
            t0 = time.time()
            _result = func(*_args) # _result is the actual result of the decorated function
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) # _args holds the actual argument of clocked, 
                                                         # args is used for display
            result = repr(_result) # result is the `str` representation of `_result` for display.
            print(fmt.format(**locals())) # using **locals() allows any local variables of clocked
                                          # to be referenced in the fmt.
            return _result # clocked replaces the decorated function, so returns whatever the function
                           # returns
        return clocked   # decorate returns clocked.
    return decorate # clock returns decorate

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

Writing clockdeco_param.py


In [12]:
!python clockdeco_param.py

[0.12325883s] snooze(0.123) -> None
[0.12315345s] snooze(0.123) -> None
[0.12327528s] snooze(0.123) -> None


In [13]:
import time
from clockdeco_param import clock

In [14]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.1231997013092041s
snooze: 0.12335586547851562s
snooze: 0.12320470809936523s


In [15]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
